In [3]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [4]:
train = pd.read_csv("C:/Class/MLproject/train3.csv")

### 파라미터 튜닝 - BayesianOptimization

In [23]:
!pip install bayesian-optimization

  Attempting uninstall: colorama
    Found existing installation: colorama 0.4.4
    Uninstalling colorama-0.4.4:
      Successfully uninstalled colorama-0.4.4


In [5]:
from xgboost import plot_importance
from xgboost import XGBRegressor
import xgboost
from sklearn.metrics import explained_variance_score
import matplotlib.pyplot as plt

X = train[['id','bus_route_id','latitude','longitude', '6~7_ride','7~8_ride',
                 '8~9_ride','9~10_ride','10~11_ride','11~12_ride','6~7_takeoff',
                 '7~8_takeoff','8~9_takeoff','9~10_takeoff','10~11_takeoff','11~12_takeoff','region_e',
       'region_n', 'region_o', 'region_s', 'region_w', 'in_out_시내',
       'in_out_시외', 'rain_0', 'rain_1', 'rain_2', 'rain_3']]


y = train[['18~20_ride']]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2)

In [33]:
!pip install colorama==0.4.4
!pip install bayesian-optimization==1.4.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bayesian-optimization 1.4.2 requires colorama>=0.4.6, but you have colorama 0.4.4 which is incompatible.


  Attempting uninstall: colorama
    Found existing installation: colorama 0.4.6
    Uninstalling colorama-0.4.6:
      Successfully uninstalled colorama-0.4.6
  Attempting uninstall: bayesian-optimization
    Found existing installation: bayesian-optimization 1.4.2
    Uninstalling bayesian-optimization-1.4.2:
      Successfully uninstalled bayesian-optimization-1.4.2


#### XGBoost 파라미터
* max_depth [default = 6] : 나무의 최대 깊이
* min_child_weight [default = 1] : child node에 필요한 인스턴스 가중치의 최소 합계
* gamma [default = 0] : 트리의 리프 노드에서 추가 파티션을 만드는 데 필요한 최소 손실 감소
* subsample [default = 1] : 훈련 인스턴스의 하위 샘플 비율
* colsample_bytree [default = 1] : 각 트리를 구성할 때 열의 하위 샘플 비율
* eta [default = 0.3] : 과적합을 방지하기 위해 업데이트에 사용되는 단계 크기 축소
* num_round : 부스팅을 위한 반복 학습 수

더 있는 듯 ..?

In [74]:
import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBRegressor
from bayes_opt import BayesianOptimization
 
dtrain = xgb.DMatrix(data=X_train, label=y_train)
dtest = xgb.DMatrix(data=X_test, label=y_test)
 

def xgb_evaluate(max_depth, gamma, colsample_bytree,min_child_weight,eta,subsample):
    params = {'eval_metric': 'rmse',
              'objective':'reg:squarederror',
              'min_child_weight': min_child_weight,
              'max_depth': int(round(max_depth)),
              'subsample': subsample,
              'eta': eta,
              'gamma': gamma,
              'colsample_bytree': colsample_bytree}
 
    cv_result = xgb.cv(params, dtrain, num_boost_round=100, nfold=3)    
    
 
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]
    # BayesianOptimization의 maximize()를 사용하기 위해, RMSE 값을 음수로 반환

In [117]:
# 하이퍼 파라미터 입력 범위 설정 및 베이지안 최적화?
xgb_bo = BayesianOptimization(xgb_evaluate, {'max_depth': (1, 15), 
                                             'gamma': (0, 1),
                                             'colsample_bytree': (0, 1.0),
                                            'min_child_weight': (0, 15),
                                            'eta':(0, 1.0),
                                            'subsample':(0, 1.0)},random_state=0)

# target(RMSE)이 최대가 되는 최적해를 찾기
xgb_bo.maximize(init_points=5, n_iter=5, acq='ei') # init_points : 처음 탐색 횟수, n_iter : 추가 반복 횟수 (연산횟수)


|   iter    |  target   | colsam... |    eta    |   gamma   | max_depth | min_ch... | subsample |
-------------------------------------------------------------------------------------------------


C:\Users\KWAK\AppData\Local\Temp\ipykernel_49592\902295529.py:10: DeprecationWarning: 
Passing acquisition function parameters or gaussian process parameters to maximize
is no longer supported, and will cause an error in future releases. Instead,
please use the "set_gp_params" method to set the gp params, and pass an instance
 of bayes_opt.util.UtilityFunction using the acquisition_function argument

  xgb_bo.maximize(init_points=5, n_iter=5, acq='ei') # init_points : 처음 탐색 횟수, n_iter : 추가 반복 횟수 (연산횟수)


| 1         | -0.274    | 0.5488    | 0.7152    | 0.6028    | 8.628     | 6.355     | 0.6459    |
| 2         | -0.2785   | 0.4376    | 0.8918    | 0.9637    | 6.368     | 11.88     | 0.5289    |
| 3         | -0.2865   | 0.568     | 0.9256    | 0.07104   | 2.22      | 0.3033    | 0.8326    |
| 4         | -0.2795   | 0.7782    | 0.87      | 0.9786    | 12.19     | 6.922     | 0.7805    |
| 5         | -0.2844   | 0.1183    | 0.6399    | 0.1434    | 14.23     | 7.828     | 0.4147    |
| 6         | -0.4757   | 1.0       | 0.0       | 1.0       | 6.177     | 7.426     | 0.0       |
| 7         | -0.8438   | 0.6278    | 0.6974    | 0.3945    | 8.484     | 6.231     | 0.03194   |
| 8         | -0.3142   | 0.6058    | 0.0129    | 0.6155    | 6.782     | 14.88     | 0.4243    |
| 9         | -0.2616   | 0.1542    | 0.1228    | 0.1202    | 14.63     | 4.575     | 0.5114    |
| 10        | -0.2701   | 0.1653    | 0.1898    | 0.4643    | 13.54     | 7.048     | 0.1907    |


In [6]:
# 최적의 파라미터 사용해서 모델 적합

xgb_model = xgboost.XGBRegressor(n_estimators=1000, learning_rate=0.08, gamma=0.1202 , subsample=0.5114 ,
                                colsample_bytree=0.1542  , max_depth=15, eta = 0.1228 , min_child_weight = 4.575 )

xgb_model.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.1542, early_stopping_rounds=None,
             enable_categorical=False, eta=0.1228, eval_metric=None,
             feature_types=None, gamma=0.1202, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.08, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=15,
             max_leaves=None, min_child_weight=4.575, missing=nan,
             monotone_constraints=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, predictor=None, ...)

In [8]:
print(xgb_model.score(X_train, y_train))
print(xgb_model.score(X_test, y_test))

0.751220995952983
0.6169200894047013


In [9]:
from dmba import regressionSummary
regressionSummary(y_test, xgb_model.predict(X_test))


Regression statistics

               Mean Error (ME) : -0.0000
Root Mean Squared Error (RMSE) : 0.2436
     Mean Absolute Error (MAE) : 0.1686


### 앙상블 (RandomForest, XGBoost, LGBM)

#### RandomForest

In [10]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=0, max_depth=8, min_samples_leaf=8, min_samples_split=8,n_estimators=200)
rf.fit(X_train, y_train)

C:\Users\KWAK\AppData\Local\Temp\ipykernel_13608\1874545960.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


RandomForestRegressor(max_depth=8, min_samples_leaf=8, min_samples_split=8,
                      n_estimators=200, random_state=0)

In [11]:
rf_predict = rf.predict(X_train)

#### XGBoost

In [12]:
xgb_predict = xgb_model.predict(X_train)

#### Lightgbm

In [95]:
!pip install lightgbm

In [13]:
import lightgbm as lgb
lgbm = lgb.LGBMRegressor(bagging_fraction=0.6,
                              early_stopping_rounds=16,
                              feature_fraction=0.73,
                              learning_rate = 0.07,
                              max_depth = 14,
                              min_child_samples = 20,
                              min_split_gain = 0.0064, 
                              n_estimators = 172,
                              num_leaves = 39,
                              boosting = "dart",
                              metric = "rmse",
                              task = "predict",
                              application = "regression",
                              eval_set = (X_test, y_test))
lgbm.fit(X_train, y_train)
lgbm_predict = lgbm.predict(X_train)

[LightGBM] [Warning] 211949 is set=, 211949= will be ignored. Current value: 211949=
[LightGBM] [Warning] 4094 is set=, 4094= will be ignored. Current value: 4094=
[LightGBM] [Warning] 0.000000 is set=, 0.000000= will be ignored. Current value: 0.000000=
[LightGBM] [Warning] 0.000000 is set=, 0.000000= will be ignored. Current value: 0.000000=
[LightGBM] [Warning] 126648 is set=, 126648= will be ignored. Current value: 126648=
[LightGBM] [Warning] 0.000000 is set=, 0.000000= will be ignored. Current value: 0.000000=
[LightGBM] [Warning] 0.000000 is set=, 0.000000= will be ignored. Current value: 0.000000=
[LightGBM] [Warning] 173077 is set=, 173077= will be ignored. Current value: 173077=
[LightGBM] [Warning] 0.000000 is set=, 0.000000= will be ignored. Current value: 0.000000=
[LightGBM] [Warning] 35691 is set=, 35691= will be ignored. Current value: 35691=
[LightGBM] [Warning] 0.000000 is set=, 0.000000= will be ignored. Current value: 0.000000=
[LightGBM] [Warning] 0.000000 is set=,

C:\Users\KWAK\anaconda3\lib\site-packages\lightgbm\callback.py:223: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


#### 각 predict 평균 내기

In [14]:
predict = pd.DataFrame(columns = ['rf','xgb','lgbm','mean'])

In [15]:
predict['rf'] = rf_predict
predict['xgb'] = xgb_predict
predict['lgbm'] = lgbm_predict
predict['mean'] = (rf_predict + xgb_predict + lgbm_predict)/3
predict['real'] = train['18~29_ride']

KeyError: '18~29_ride'

In [107]:
predict

,rf,xgb,lgbm,mean
0,0.574263,0.371870,0.545086,0.497073
1,0.359728,0.099094,0.183318,0.214046
2,0.138329,0.152509,0.116539,0.135793
3,0.129240,0.360760,0.205668,0.231889
4,0.058800,0.046879,0.029344,0.045008
...,...,...,...,...
290791,0.051301,0.002663,0.017519,0.023828
290792,0.431791,0.367938,0.404935,0.401555
290793,0.150508,0.335344,0.107142,0.197664
290794,0.127986,0.085806,0.100264,0.104685


NameError: name 'predict' is not defined

행 갯수가 왜 다를까 .. 